In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


# sigmoid 함수
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata, lr, cnt):  # xdata, tdata => numpy.array(...)
        
        self.name = gate_name
        
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 가중치 W, 바이어스 b 초기화 - 일반적인 방법으로 변경 필요
        self.W = np.random.rand(self.xdata.shape[1], 1)  # weight, 2 X 1 matrix
        self.b = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = lr
        
        # 반복횟수 초기화
        self.iteration_count = cnt
        
    # 손실함수
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )      
    
    # 손실 값 계산
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        step_count = int(self.iteration_count * 0.1)
        
        f = lambda x : self.loss_func()
        
        print("Initial error value = ", self.error_val())
        
        start_time = datetime.now()
        
        for step in  range(self.iteration_count):
            
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % step_count == 0):
                print("step = ", step, "error value = ", self.error_val())
                
        end_time = datetime.now()
        
        print('\nElapsed Time => ', end_time - start_time)
                
    # 미래 값 예측 함수
    def predict(self, input_data):
        
        z = np.dot(input_data, self.W) + self.b
        y = sigmoid(z)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(test_xdata)
        
        return accuracy_val

In [3]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

AND_obj = LogicGate("AND_GATE", xdata, tdata, 1e-2, 20001)

AND_obj.train()

Initial error value =  3.05008612765222
step =  0 error value =  3.024703808021531
step =  2000 error value =  0.65353732693501
step =  4000 error value =  0.3878510180991557
step =  6000 error value =  0.27367283841531376
step =  8000 error value =  0.21051373837990545
step =  10000 error value =  0.17062556311238397
step =  12000 error value =  0.14323692461256043
step =  14000 error value =  0.12331012684615866
step =  16000 error value =  0.10818267800882005
step =  18000 error value =  0.09631873275861358
step =  20000 error value =  0.08677183565894185

Elapsed Time =>  0:00:08.948537


In [4]:
# AND Gate prediction
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

accuracy_ret = AND_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

AND_GATE 

Accuracy =>  1.0


In [5]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata, 1e-2, 20001)

OR_obj.train() 

Initial error value =  1.7156491192175976
step =  0 error value =  1.7123523635800009
step =  2000 error value =  0.4203174635304236
step =  4000 error value =  0.2290602471735538
step =  6000 error value =  0.15543635320938465
step =  8000 error value =  0.11706869628596822
step =  10000 error value =  0.09368122350152443
step =  12000 error value =  0.07798628608070646
step =  14000 error value =  0.06674609069615747
step =  16000 error value =  0.0583097917098034
step =  18000 error value =  0.05174976770114202
step =  20000 error value =  0.04650561427565133

Elapsed Time =>  0:00:21.946231


In [6]:
# OR Gate prediction
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

accuracy_ret = OR_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

OR_GATE 

Accuracy =>  1.0


In [7]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata, 1e-2, 20001)

NAND_obj.train()

Initial error value =  3.087834163921827
step =  0 error value =  3.075780968015838
step =  2000 error value =  0.6758006316108076
step =  4000 error value =  0.39578318388523837
step =  6000 error value =  0.2776918427082521
step =  8000 error value =  0.21292146335203435
step =  10000 error value =  0.17222115753215184
step =  12000 error value =  0.14436854909773306
step =  14000 error value =  0.124152823974862
step =  16000 error value =  0.10883372132665074
step =  18000 error value =  0.09683635158434373
step =  20000 error value =  0.0871929518055678

Elapsed Time =>  0:00:17.106716


In [8]:
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

accuracy_ret = NAND_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

NAND_GATE 

Accuracy =>  1.0


In [9]:
# XOR 을 NAND + OR => AND 조합으로 계산함

input_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

s1 = []    # NAND 출력
s2 = []    # OR 출력

new_input_data = []  # AND 입력
final_output = []    # AND 출력

for index in range(len(input_data)):
    
    s1 = NAND_obj.predict(input_data[index])  # NAND 출력
    s2 = OR_obj.predict(input_data[index])    # OR 출력
    
    new_input_data.append(s1[-1])    # AND 입력
    new_input_data.append(s2[-1])    # AND 입력
    
    (sigmoid_val, logical_val) = AND_obj.predict(np.array(new_input_data))
    
    final_output.append(logical_val)    # AND 출력, 즉 XOR 출력    
    new_input_data = []    # AND 입력 초기화


for index in range(len(input_data)):    
    print(input_data[index], " = ", final_output[index], end='')
    print("\n")

[0 0]  =  0

[0 1]  =  1

[1 0]  =  1

[1 1]  =  0

